In [97]:
from nltk.corpus import treebank
import numpy as np

In [118]:
data = treebank.tagged_words()
sentences = treebank.tagged_sents()
max_length = 271
max_tag = 46


In [99]:
def make_index(s):
    dict_s = dict.fromkeys(s)
    k = 0
    for i in dict_s:
        dict_s[i] = k
        k +=1
    return dict_s

In [100]:
def data_inf(data):
    word = set()
    pos = set()
    pairs = set()
    for pair in data:
        word.add(pair[0])
        pos.add(pair[1])
        pairs.add(pair)
    word = make_index(word)
    pos = make_index(pos)
    pairs = make_index(pairs)
    return word,pos,pairs

In [101]:
word_index,pos_index,pairs_index = data_inf(data)

In [102]:
def presentation_word(word, word_index):
    l = len(word_index)
    index = word_index[word]
    word_feat = np.zeros(l)
    word_feat[index] = 1
    return word_feat

In [103]:
def presentation_pos(pos, pos_index):
    l = len(pos_index)
    index = pos_index[pos]
    pos_feat = np.zeros(l)
    pos_feat[index] = 1
    return pos_feat

In [104]:
def presentation_pair(pair, pairs_index):
    l = len(pairs_index)
    index = pairs_index[pair]
    pair_feat = np.zeros(l)
    pair_feat[index] = 1
    return pair_feat

In [112]:
def position(position):
    position_feat = np.zeros(max_length)
    position_feat[position] = 1
    return position_feat

In [150]:
def neighbors_pos(position, sentence_pos):
    last_pos_feat = np.zeros(max_tag+1)
    last_two_pos_feat = np.zeros(max_tag+1)
    next_pos_feat = np.zeros(max_tag+1)
    next_two_pos_feat = np.zeros(max_tag+1)
    #length of the sentence
    max_length_sentence = len(sentence_pos)
    #pos for neighbors
    if position==0:
        last_pos_feat[-1] = 1
        last_two_pos_feat[-1] = 1
        if max_length_sentence==1:
            next_pos_feat[-1]=1
            next_two_pos_feat[-1]=1
        elif max_length_sentence == 2:
            next_pos_feat[sentence_pos[1]] = 1
            next_two_pos_feat[-1] = 1
        else:
            next_pos_feat[sentence_pos[1]]=1
            next_two_pos_feat[sentence_pos[2]] =1
    elif position == 1:
        last_pos_feat[sentence_pos[0]]=1
        last_two_pos_feat[-1]=1
        if max_length_sentence==2:
            next_pos_feat[-1]=1
            next_two_pos_feat[-1]=1
        elif max_length==3:
            next_pos_feat[sentence_pos[2]]=1
            next_two_pos_feat[-1]=1
        else:
            next_pos_feat[sentence_pos[position+1]]=1
            next_two_pos_feat[sentence_pos[position+2]]=1
    else:
        if position + 1 == max_length_sentence:
            next_pos_feat[-1]=1
            next_two_pos_feat[-1]=1
            last_pos_feat[sentence_pos[position-1]]=1
            last_two_pos_feat[sentence_pos[position-2]]=1
        elif position + 2 == max_length_sentence:
            next_pos_feat[sentence_pos[position+1]]=1
            next_two_pos_feat[-1]=1
            last_pos_feat[sentence_pos[position-1]]=1
            last_two_pos_feat[sentence_pos[position-2]]=1
        else:
            next_pos_feat[sentence_pos[position+1]]=1
            next_two_pos_feat[sentence_pos[position+2]]=1
            last_pos_feat[sentence_pos[position-1]]=1
            last_two_pos_feat[sentence_pos[position-2]]=1
    return next_pos_feat,next_two_pos_feat,last_pos_feat,last_two_pos_feat
            
    

In [151]:
sentences[0]

[(u'Pierre', u'NNP'),
 (u'Vinken', u'NNP'),
 (u',', u','),
 (u'61', u'CD'),
 (u'years', u'NNS'),
 (u'old', u'JJ'),
 (u',', u','),
 (u'will', u'MD'),
 (u'join', u'VB'),
 (u'the', u'DT'),
 (u'board', u'NN'),
 (u'as', u'IN'),
 (u'a', u'DT'),
 (u'nonexecutive', u'JJ'),
 (u'director', u'NN'),
 (u'Nov.', u'NNP'),
 (u'29', u'CD'),
 (u'.', u'.')]

In [152]:
w=[]
t=[]
t_index=[]
for i in sentences[0]:
    w.append(i[0])
    t_index.append(pos_index[i[1]])
    t.append(i[1])

In [153]:
t

[u'NNP',
 u'NNP',
 u',',
 u'CD',
 u'NNS',
 u'JJ',
 u',',
 u'MD',
 u'VB',
 u'DT',
 u'NN',
 u'IN',
 u'DT',
 u'JJ',
 u'NN',
 u'NNP',
 u'CD',
 u'.']

In [167]:
feat = neighbors_pos(position=18,sentence_pos=t_index)

IndexError: list index out of range

In [168]:
for i in feat:
    print np.argmax(i)

46
46
33
25


In [169]:
t_index

[25, 25, 4, 33, 24, 8, 4, 38, 44, 11, 15, 36, 11, 8, 15, 25, 33, 18]

In [163]:
feat

(array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]),
 array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.]),
 array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]),
 array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,